In [1]:
import os
sys.path.append(os.path.abspath("../../"))
from iROIEstimator import iROIEstimator
%matplotlib inline
os.getcwd()

'/Users/alvaradojo/Documents/Github/iROIEstimator/scripts/notebook/results'

In [2]:


estimator = iROIEstimator("angular/angular")
estimator.execute()

/Users/alvaradojo/Documents/Github/iROIEstimator/scripts/notebook/results
INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.

angular - BUG - Module_CC prediction count: 82
   Project      Model Task  R_Squared  R_Squared_Adj     MAE        MSE  \
0  angular  Module_CC  BUG        0.8           0.79  179.81  217762.56   

     RMSE  PRED_25  PRED_50  T_Records  
0  466.65     0.34     0.54        326  
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.

angular - BUG - Module_EC prediction count: 82
   Project      Model Task  R_Squared  R_Squared_Adj    MAE       MSE    RMSE  \
0  angular  Module_EC  BUG        0.5           0.48  35.17  50520.53  224.77   

   PRED_25  PRED_50  T_Records  
0     0.41     0.63        326  
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override th

KeyboardInterrupt: 